# Equity Factor Backtesting Demo

This notebook demonstrates how to use the equity factor backtesting framework to:
1. Load market data
2. Calculate factor scores
3. Run backtests
4. Analyze performance


In [ ]:
import sys
import os

# Add src to path
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import our framework
from equity_backtesting import BacktestEngine, FactorLibrary, DataProvider, PerformanceAnalytics
from equity_backtesting.backtesting.engine import BacktestConfig, StrategyConfig, create_default_backtest

# Set up plotting
plt.style.use('seaborn-v0_8')
%matplotlib inline

print("Framework imported successfully!")

## 1. Create a Simple Momentum Strategy

In [ ]:
# Define our strategy: equal-weighted momentum factors
strategy_factors = {
    'momentum_3m': 0.5,
    'momentum_6m': 0.5
}

# Create backtest configuration
backtest_config, strategy_config = create_default_backtest(
    factors=strategy_factors,
    start_date="2020-01-01",
    end_date="2023-12-31"
)

print(f"Universe size: {len(backtest_config.universe)}")
print(f"Strategy factors: {strategy_config.factors}")

## 2. Run the Backtest

In [ ]:
# Initialize backtest engine
engine = BacktestEngine(backtest_config)

# Run the backtest
print("Running backtest... (this may take a few minutes)")
results = engine.run_backtest(strategy_config)

print(f"Backtest completed!")
print(f"Portfolio returns shape: {results['portfolio_returns'].shape}")
print(f"First few returns:")
print(results['portfolio_returns'].head())

## 3. Performance Analysis

In [ ]:
# Initialize performance analytics
analytics = PerformanceAnalytics(results)

# Generate performance report
performance_report = analytics.generate_performance_report()
print("Performance Metrics:")
print("=" * 50)
print(performance_report)

## 4. Visualizations

In [ ]:
# Plot cumulative returns
fig = analytics.plot_cumulative_returns(figsize=(12, 6))
plt.show()

In [ ]:
# Plot drawdown
fig = analytics.plot_drawdown(figsize=(12, 4))
plt.show()

In [ ]:
# Risk-return scatter
fig = analytics.plot_risk_return_scatter(figsize=(8, 6))
plt.show()

## 5. Compare Different Strategies

In [ ]:
# Let's compare different factor combinations
strategies_to_test = {
    'Momentum Only': {'momentum_3m': 1.0},
    'Low Vol Only': {'low_volatility': 1.0},
    'Value Only': {'value': 1.0},
    'Balanced': {'momentum_3m': 0.4, 'low_volatility': 0.3, 'value': 0.3}
}

strategy_results = {}

for strategy_name, factors in strategies_to_test.items():
    print(f"Testing {strategy_name}...")
    
    # Create configs
    test_backtest_config, test_strategy_config = create_default_backtest(
        factors=factors,
        start_date="2021-01-01",  # Shorter period for faster testing
        end_date="2023-12-31"
    )
    
    # Run backtest
    test_engine = BacktestEngine(test_backtest_config)
    test_results = test_engine.run_backtest(test_strategy_config)
    
    # Store results
    strategy_results[strategy_name] = test_results['portfolio_returns']

print("All strategies tested!")

In [ ]:
# Compare cumulative returns
fig, ax = plt.subplots(figsize=(12, 8))

for strategy_name, returns in strategy_results.items():
    cumulative = (1 + returns).cumprod()
    ax.plot(cumulative.index, cumulative.values, label=strategy_name, linewidth=2)

ax.set_title('Strategy Comparison: Cumulative Returns', fontsize=16, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Cumulative Return', fontsize=12)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

# Format y-axis as percentage
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y-1)))

plt.tight_layout()
plt.show()

In [ ]:
# Calculate summary metrics for each strategy
summary_metrics = []

for strategy_name, returns in strategy_results.items():
    returns = returns.dropna()
    
    total_return = (1 + returns).prod() - 1
    volatility = returns.std() * np.sqrt(252)
    sharpe = returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0
    
    # Max drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative - running_max) / running_max
    max_dd = drawdown.min()
    
    summary_metrics.append({
        'Strategy': strategy_name,
        'Total Return': f"{total_return:.1%}",
        'Volatility': f"{volatility:.1%}",
        'Sharpe Ratio': f"{sharpe:.2f}",
        'Max Drawdown': f"{max_dd:.1%}"
    })

summary_df = pd.DataFrame(summary_metrics)
print("Strategy Comparison Summary:")
print("=" * 60)
print(summary_df.to_string(index=False))

## 6. Factor Analysis

In [ ]:
# Let's examine individual factors
factor_lib = FactorLibrary()
data_provider = DataProvider()

print("Available factors:")
for category in ['momentum', 'value', 'low_volatility', 'size', 'quality']:
    factors = factor_lib.get_factors_by_category(category)
    print(f"  {category.title()}: {factors}")

## 7. Export Results

In [ ]:
# Save results to CSV
output_dir = '../data'
os.makedirs(output_dir, exist_ok=True)

# Save portfolio returns
results['portfolio_returns'].to_csv(f"{output_dir}/portfolio_returns.csv")

# Save performance metrics
performance_report.to_csv(f"{output_dir}/performance_metrics.csv")

# Save strategy comparison
summary_df.to_csv(f"{output_dir}/strategy_comparison.csv", index=False)

print("Results saved to data/ directory")

## Conclusion

This notebook demonstrated the basic usage of the equity factor backtesting framework:

1. **Strategy Definition**: We defined factor-based strategies with different weights
2. **Backtesting**: We ran comprehensive backtests with transaction costs and rebalancing
3. **Performance Analysis**: We calculated key risk and return metrics
4. **Visualization**: We created charts to understand strategy performance
5. **Comparison**: We compared different factor combinations

### Next Steps

- Experiment with different factor combinations
- Try different rebalancing frequencies
- Implement custom factors
- Add sector neutralization
- Optimize factor weights using machine learning
